Mainly serve as a template for future strategies. The template use simple moving average (SMA) of different timeframes to generate buy and sell signals. The higher timeframe SMA is used to generate the trend, and the lower timeframe SMA is used to generate the entry and exit points.

In [ ]:
from typing_extensions import Dict
import optuna
import pandas as pd
import numpy as np
import talib as ta
# --- Our Imports ---
from AlgoTrade.Utils.DataManager import DataManager
from AlgoTrade.Config.BacktestConfig import BacktestConfig
from AlgoTrade.Backtester.BacktestRunner import BacktestRunner
from AlgoTrade.Optimizer.StudyRunner import run_optimization

In [2]:
dm = DataManager(name="bitget")
ohlcv_data_s = dm.from_local("ADA/USDT:USDT", "15m", "2024-12-01")
ohlcv_data_l = dm.from_local("ADA/USDT:USDT", "4h", "2025-01-01")

Loading data from d:\ComputerScience\Trading\Quant2\data\bitget\ADA_USDT_USDT_15m.csv...
Load completed successfully. Data shape: (16960, 6)
Loading data from d:\ComputerScience\Trading\Quant2\data\bitget\ADA_USDT_USDT_4h.csv...
Load completed successfully. Data shape: (874, 5)


d:\ComputerScience\Trading\Quant2\StrategyLab\Utils\DataManager.py:291: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data.index = pd.to_datetime(data.index)


In [3]:
def pre_process_data(df1: pd.DataFrame, df2: pd.DataFrame):
    df1["datetime"] = df1.index
    df1.reset_index(drop=True, inplace=True)
    df2["datetime"] = df2.index
    df2.reset_index(drop=True, inplace=True)
    print("Data preprocessed.")
    return df1, df2

In [4]:
def compute_indicators_df1(
    df1: pd.DataFrame,
    period_df1: int,
):
    df1["MA"] = ta.SMA(df1["close"], timeperiod=period_df1)
    df1["STDDEV"] = ta.STDDEV(df1["close"], timeperiod=14)
    df1["ATR"] = ta.ATR(df1["high"], df1["low"], df1["close"], timeperiod=14)
    df1["SD_ATR_Spread"] = df1["STDDEV"] - df1["ATR"]
    print("Indicators computed (df1).")
    return df1


def compute_indicators_df2(df2: pd.DataFrame, period_df2: int):
    df2["MA"] = ta.SMA(df2["close"], timeperiod=period_df2)
    trend = np.where(df2["close"] > df2["MA"], 1, -1)
    df2["trend"] = pd.Series(trend).shift(1).fillna(0)
    print("Indicators computed (df2).")
    return df2


def merge_htf_into_ltf(df_ltf, df_htf, suffix):
    df_ltf = df_ltf.copy()
    df_htf = df_htf.copy()
    df_ltf["datetime"] = pd.to_datetime(df_ltf["datetime"])
    df_htf["datetime"] = pd.to_datetime(df_htf["datetime"])
    df_ltf.set_index("datetime", inplace=True)
    df_htf.set_index("datetime", inplace=True)
    df_htf = df_htf.shift(1)
    df_htf_resampled = df_htf.reindex(df_ltf.index, method="ffill")
    df_merged = df_ltf.join(df_htf_resampled, rsuffix=suffix)
    df_merged.dropna(inplace=True)
    return df_merged.reset_index()

In [5]:
df1, df2 = pre_process_data(ohlcv_data_s, ohlcv_data_l)
df1 = compute_indicators_df1(df1, 90)
df2 = compute_indicators_df2(df2, 14)

Data preprocessed.
Indicators computed (df1).
Indicators computed (df2).


In [6]:
merge_df = merge_htf_into_ltf(df1, df2, suffix="_4h")


In [7]:
def compute_signals(merged_df: pd.DataFrame):
    signal = np.where(
        (merged_df["trend"] == 1)
        & (merged_df["close"] > merged_df["MA"])
        & (merged_df["SD_ATR_Spread"] > 0),
        1,
        np.where(
            (merged_df["trend"] == -1)
            & (merged_df["close"] < merged_df["MA"])
            & (merged_df["SD_ATR_Spread"] > 0),
            -1,
            0,
        ),
    )
    merged_df["signal"] = pd.Series(signal, index=merged_df.index).shift(1).fillna(0)
    return merged_df

In [8]:
solution = compute_signals(merge_df)

# Set datetime to index to use the backtest

In [9]:
solution.set_index('datetime', inplace=True)

# Select Position Sizing Method

In [10]:
config = BacktestConfig(
    initial_balance=100.0,
    trading_mode="Cross",
    leverage=100,
    position_sizing_method="PercentBalance",
    percent_balance_pct=0.1,  # <-- UPDATED
    exit_on_signal_0=True,
    # General SL/TP can be used as a fallback
    stop_loss_pct=0.02,
    take_profit_pct=0.02,
)

In [11]:
runner = BacktestRunner(config=config, data=solution)
analysis = runner.run()
analysis.print_metrics()

Starting backtest...
Backtest finished.

--- Backtest Results ---
Period: [2025-01-03] -> [2025-05-26]
Initial Balance:        100.00
Final Balance:          0.00
ROI:                    -100.00%
Profit:                 154.39
Loss:                   254.39
Sharpe Ratio:           -6.46
Sortino Ratio:          -9.81
Calmar Ratio:           -1.00
Max Drawdown:           100.00%
Total Trades:           671
Total Good Trades:      269
Total Bad Trades:       402
Avg PnL Good Trades:    93.82%
Avg PnL Bad Trades:     -83.80%
Win Rate:               40.09%
Loss Rate:              59.91%
Profit Factor:          0.61


In [12]:
ex = analysis.results_df

# Select All Position Sizing Method

In [17]:
base_config = BacktestConfig(
    initial_balance=100.0,
    trading_mode="Cross",
    leverage=10,
    # --- Parameters for all sizing methods ---
    # For PercentBalance
    percent_balance_pct=0.1,
    # For FixedAmount
    fixed_amount_size=20.0,  # $20 margin per trade
    stop_loss_pct=0.02,
    take_profit_pct=0.02,
    # For AtrVolatility
    atr_volatility_risk_pct=0.02,
    atr_volatility_period=14,
    atr_volatility_multiplier=2.5,
    # For KellyCriterion
    kelly_criterion_lookback=50,
    kelly_criterion_fraction=0.5,
    # For AtrBands
    atr_bands_risk_pct=0.02,
    atr_bands_period=14,
    atr_bands_multiplier=2.0,
    atr_bands_risk_reward_ratio=1.5,
)

In [ ]:
from AlgoTrade.Backtester import ComparativeRunner as cr

In [19]:
res = cr.run_comparative_analysis(base_config, solution)


--------------------------------------------------
--- Running Backtest for: PercentBalance ---
--------------------------------------------------
Starting backtest...
Backtest finished.

--------------------------------------------------
--- Running Backtest for: FixedAmount ---
--------------------------------------------------
Starting backtest...
Backtest finished.

--------------------------------------------------
--- Running Backtest for: AtrVolatility ---
--------------------------------------------------
Starting backtest...
Backtest finished.

--------------------------------------------------
--- Running Backtest for: KellyCriterion ---
--------------------------------------------------
Starting backtest...
Backtest finished.

--------------------------------------------------
--- Running Backtest for: AtrBands ---
--------------------------------------------------
Starting backtest...
Backtest finished.


In [20]:
cr.print_comparison_report(res)


                    Comprehensive Backtest Comparison Report
Period: [2025-01-03] -> [2025-05-26]
Method                  PercentBalance FixedAmount AtrVolatility  \
initial_balance                $100.00     $100.00       $100.00   
final_balance                   $56.41      $19.95        $84.24   
roi_pct                        -43.59%     -80.05%       -15.76%   
total_profits                  $272.98     $376.31       $313.21   
total_losses                   $316.86     $456.36       $329.62   
sharpe_ratio                    -1.906      -1.625        -0.318   
sortino_ratio                   -2.953      -1.653        -0.703   
calmar_ratio                    -1.551      -1.225        -1.148   
max_drawdown_equity             49.53%      80.28%        30.87%   
total_trades                       429         209           394   
total_good_trades                  210          98           208   
total_bad_trades                   219         111           186   
avg_pnl_pct_good_

# Optimizer



In [9]:
def generate_signals_for_trial(
    trial: optuna.trial.Trial, data_dict: Dict[str, pd.DataFrame]
) -> pd.DataFrame:
    """
    This function is passed to the optimizer. It defines the parameters to tune
    and returns a DataFrame with a 'signal' column.
    """
    # 1. Define the parameters you want to optimize for this strategy
    ltf_ma_period = trial.suggest_int("ltf_ma_period", 20, 100, step=5)
    htf_ma_period = trial.suggest_int("htf_ma_period", 5, 50, step=1)
    # You could also tune other things, e.g., a volatility filter threshold
    # min_spread = trial.suggest_float("min_spread", -0.0001, 0.0001)

    # 2. Get the data
    df1, df2 = pre_process_data(
        data_dict["ltf_data"].copy(), data_dict["htf_data"].copy()
    )

    # 3. Calculate indicators using the trial's parameters
    df1 = compute_indicators_df1(df1, period_df1=ltf_ma_period)
    df2 = compute_indicators_df2(df2, period_df2=htf_ma_period)

    # 4. Merge and compute final signal
    merged_df = merge_htf_into_ltf(df1, df2, suffix="_4h")
    solution_df = compute_signals(merged_df)

    # 5. Set index and return
    solution_df.set_index("datetime", inplace=True)
    return solution_df

In [10]:
data_dictionary = {
    "ltf_data": ohlcv_data_s,
    "htf_data": ohlcv_data_l,
}

fixed_config = BacktestConfig(
    initial_balance=100.0,
    trading_mode="Cross",
    leverage=10,
    position_sizing_method="AtrBands",
    atr_bands_risk_pct=0.01,
    atr_bands_period=14,
    atr_bands_multiplier=2.0,
    atr_bands_risk_reward_ratio=1.5,
    exit_on_signal_0=False,
    allow_reverse_trade=False,
)

In [ ]:
print("\n--- Running Strategy Optimization ---")
run_optimization(
    data_dict=data_dictionary,
    config=fixed_config,
    strategy_function=generate_signals_for_trial,
    n_trials=10,
    metric_to_optimize="sharpe_ratio"
)
